# Supervised Learning Project

## EDA

Before, the EDA is carried on, the data needs to be filtered by extracting the training sessions from the `full-data` folder which contains all the training sessions from different disciplines and lengths. We are interested only in sessions that correspond to trail running and have a duration of at least 3 hours. We parse the training sessions in json format and store the metrics of interest in CSV format in the `long-tr-data`

In [2]:
# Install rdflib to use isodate
%pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 6.0 MB/s eta 0:00:00a 0:00:01


In [ ]:
import pandas as pd
import os
import json
import isodate
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Create the output directory if it doesn't exist
# output_dir = "./data/long-tr-data"
output_dir = "/home/eaguayo/workspace/ml-project/data/long-tr-data"
os.makedirs(output_dir, exist_ok=True)

# Iterate through all files in the ./full-data folder
# input_dir = "./data/full-data"
input_dir = "/home/eaguayo/workspace/ml-project/data/full-data"
count = 0


def process_file(file_name):
    file_path = f"{input_dir}/{file_name}"
    try:
        with open(file_path, "r") as f:
            data = json.load(f)

        exercise = data.get("exercises", [])[0]

        # 1. check if the sport corresponds to trail running and if the duration is greater than 3 hours
        sport = exercise.get("sport")
        duration_iso = exercise.get("duration")
        duration = isodate.parse_duration(duration_iso).total_seconds()

        if sport != "TRAIL_RUNNING" or duration < 3 * 3600:
            return False

        # --- 2. Extract available data ---
        print(f"Processing file: {file_name}")
        start_time = time.time()

        sample_features = [
            "heartRate",
            "altitude",
            "distance",
            "temperature",
            "cadence",
            "speed",
        ]
        samples = exercise.get("samples", {})

        # Initialize main dataframe with heart rate samples
        df = pd.DataFrame(
            [
                {
                    "timestamp": pd.to_datetime(sample["dateTime"]),
                    "heartRate": sample["value"],
                }
                for sample in samples.get("heartRate", [])
            ]
        )

        # Process and merge other sample types
        for sample_feature in sample_features[1:]:  # Skip heartRate (already in df)
            sample_data = samples.get(sample_feature, [])
            if not sample_data:
                continue
            temp_df = pd.DataFrame(
                [
                    {
                        "timestamp": pd.to_datetime(sample["dateTime"]),
                        sample_feature: sample["value"],
                    }
                    for sample in sample_data
                ]
            )
            df = pd.merge(df, temp_df, on="timestamp", how="left")

        # Save the dataframe to a CSV file
        output_file_name = f"{output_dir}/{file_name.replace('.json', '.csv')}"
        df.to_csv(output_file_name, index=False)
        end_time = time.time()
        print(f"Saved processed data to: {output_file_name}")
        print(f"Processing time for {file_name}: {end_time - start_time:.2f} seconds")
        return True
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_name}")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file: {file_name}")
    except Exception as e:
        print(f"An unexpected error occurred in file: {file_name} with error {e}")
    return False


# Get the list of files
files = os.listdir(input_dir)

# Process files sequentially
for file_name in tqdm(files):
    if process_file(file_name):
        count += 1

# Process files in parallel with a progress bar
# TODO: Does not seem to work well from vscode ipynb with local kernel 
#   or remote jupyter server runtime. Maybe split to a separate .py script file.
# with ThreadPoolExecutor(max_workers=8) as executor:
#     futures = {executor.submit(process_file, file_name): file_name for file_name in files}
#     for future in tqdm(as_completed(futures), total=len(futures)):
#         if future.result():
#             count += 1

print(f"Processed {count} files.")

  0%|          | 7/1468 [00:00<00:24, 58.91it/s]

Processing file: training-session-2023-03-18-7606603203-e51b3f6f-5e70-4fbf-b306-060ab13c8302.json


  1%|          | 13/1468 [01:10<2:36:22,  6.45s/it]

Saved processed data to: /home/eaguayo/workspace/ml-project/data/long-tr-data/training-session-2023-03-18-7606603203-e51b3f6f-5e70-4fbf-b306-060ab13c8302.csv
Processing time for training-session-2023-03-18-7606603203-e51b3f6f-5e70-4fbf-b306-060ab13c8302.json: 69.96 seconds
Processing file: training-session-2021-05-16-6063407634-d69f9ab5-57fc-450a-be50-8a1d947c9014.json
